In [1]:
#Data-analytiikan ohjelmointikielet
#3. Datan muokkaus ja siivoamine
#Julia-esimerkit


#Ladataan tilastokeskuksen data. Haetaan data JSON-muodossa suoraan Tilastokeskuksen PXWeb-palvelun tarjoaman
#rajapinnan kautta. Asenna HTTP ja JSON -paketit ennen koodin suorittamista.

#PxWebin käyttöliittymän osoite: https://pxdata.stat.fi/PxWeb/pxweb/fi/StatFin/StatFin__aku/statfin_aku_pxt_12dz.px/




using HTTP, JSON, DataFrames

url = "https://pxdata.stat.fi/PxWeb/api/v1/fi/StatFin/aku/statfin_aku_pxt_12dz.px" #Saat urlin web-käyttöliittymästä. 

body = """
{
  "query": [
    {
      "code": "Ikä",
      "selection": {
        "filter": "item",
        "values": [
          "18-24",
          "25-34",
          "35-44",
          "45-54",
          "55-64"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

"""

response = HTTP.post(url,
    ["Content-Type" => "application/json"],  # headerit
    body
)

data = JSON.parse(String(response.body))

println(typeof(data))               # Tuloksesta huomataan, että data on sanakirja-tyyppinen
println(keys(data))                 # Tarkistetaan avaimet



Dict{String, Any}
["label", "role", "id", "value", "version", "updated", "note", "class", "dimension", "size", "extension", "source"]


In [2]:
println(data) #Tutkitaan nopealla silmäyksellä ladatun datan rakenne.

Dict{String, Any}("label" => "Aikuiskoulutukseen osallistuminen (ml. työhön tai ammattiin liittyvä, henkilöstökoulutus sekä muu kuin työhön liittyvä aikuiskoulutus) ikäluokittain muuttujina Vuosi, Ikä ja Tiedot", "role" => Dict{String, Any}("time" => Any["Vuosi"], "metric" => Any["Tiedot"]), "id" => Any["Vuosi", "Ikä", "Tiedot"], "value" => Any[204552, 43, 97831, 36, 82445, 34, 49301, 17, 416027, 55, 329104, 50, 285334, 50, 127314, 18, 481846, 57, 373050, 48, 338924, 52, 172371, 21, 291068, 48, 234914, 44, 209508, 48, 111111, 18, 132963, 25, 57283, 27, 52524, 37, 85202, 16, 183534, 42, 50555, 27, 37360, 33, 36170, 18, 381866, 53, 260675, 44, 204640, 50, 126507, 19, 425144, 54, 325990, 45, 264282, 53, 147087, 19, 404908, 54, 317436, 47, 261089, 57, 131040, 17, 160102, 31, 77434, 33, 63460, 51, 77844, 15, 225413, 49, 77432, 41, 62174, 46, 39338, 19, 409527, 62, 298999, 56, 241792, 57, 121145, 20, 474888, 62, 388827, 56, 322034, 61, 122872, 17, 463366, 56, 379929, 52, 307692, 57, 139374, 

In [3]:
#Dataa on todella hankala lukea tässä muodossa. Avataan sen rakennetta vähän. Kyseessä on moniulotteinen taulukko eli tietokuutio 
#(data cube). Tässä data on järjestelty kolmen muuttujan suhteen (kolmiuloitteinen): Vuosi, ikäryhmä ja tiedot. Jokainen havainto löytyy 
#kolmiuloitteisessa avaruudessa muuttujien risteyspisteestä. Julia esittää sen dict-muotoisena.

#Koska moniulotteista dataa on hankala hahmottaa, muutetaan se dataframe-muotoon. Sitä varten on tunnettava Tilastokeskuksen PxWebin tapa
#esittää data. 

#TEHTÄVÄ Selvitä tekoälyn avulla miten data PxWebissä säilötään. Kun saat sen selville, niin alla oleva koodikin avautuu paremmin. Tehtävää
#ei tarvitse palauttaa. 


# Haetaan jokaisen dimension (ulottuvuus) kohdalla arvojen oikea järjestys. Jotta voidaan tehdä dataframe oikein, pitää tietää tarkalleen
#missä järjestyksessä vuodet, ikäryhmät ja tiedot esiintyvät tietokuutiossa. 

iv = data["dimension"]["Vuosi"]["category"]["index"]   #sanakirja, jota käytetään vuodet-dimension järjestämiseen. 
ii = data["dimension"]["Ikä"]["category"]["index"] #sanakirja, jota käytetään iat-dimension järjestämiseen
it = data["dimension"]["Tiedot"]["category"]["index"] #sanakirja, jota käytetään tiedot-dimension järjestämiseen. 

# Järjestetään kaikki dimensiot indeksin mukaan. Keys hakee kaikki indeksit ja sort järjestää ne listaksi ehdon mukaan. 
vuodet = sort(collect(keys(iv)); by = k -> iv[k])
iat  = sort(collect(keys(ii)); by = k -> ii[k])
tiedot  = sort(collect(keys(it)); by = k -> it[k])

arvot = data["value"]  # Arvot yhdessä vektorissa

# Luodaan kaikki datan yhdistelmät järjestyksessä: Vuosi -> Ikä -> Tiedot (Tiedot vaihtuu nopeimmin)
yhdistelmat = [(v,i,t) for v in vuodet for i in iat for t in tiedot]

# 4) Rakenna DataFrame
df = DataFrame(Vuosi=[c[1] for c in yhdistelmat],
               Ikä  =[c[2] for c in yhdistelmat],
               Tiedot=[c[3] for c in yhdistelmat],
               Arvo = arvot)



df


Row,Vuosi,Ikä,Tiedot,Arvo
,String,String,String,Any
1,1990,18-24,akuo_lkm,204552
2,1990,18-24,akuo_pros,43
3,1990,18-24,tyoo_lkm,97831
4,1990,18-24,tyoo_pros,36
5,1990,18-24,tyan_lkm,82445
6,1990,18-24,tyan_pros,34
7,1990,18-24,muuo_lkm,49301
8,1990,18-24,muuo_pros,17
9,1990,25-34,akuo_lkm,416027


In [4]:
#Huomataan, että Tiedot-sarakkeessa ei ole muuttujalle selkeää suomenkielistä selitettä. Haetaan koodeja 
#vastaavat selitteet. 

labels = data["dimension"]["Tiedot"]["category"]["label"]
df.Tiedot = [get(labels, k, k) for k in df.Tiedot]

df

Row,Vuosi,Ikä,Tiedot,Arvo
,String,String,String,Any
1,1990,18-24,"Aikuiskoulutukseen osallistuneet, lkm",204552
2,1990,18-24,"Aikuiskoulutukseen osallistuneet, %",43
3,1990,18-24,"Työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, lkm",97831
4,1990,18-24,"Työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, %",36
5,1990,18-24,"Työnantajan tukemaan koulutukseen (henkilöstökoulutukseen) osallistuneet, lkm",82445
6,1990,18-24,"Työnantajan tukemaan koulutukseen (henkilöstökoulutukseen) osallistuneet, %",34
7,1990,18-24,"Muuhun kuin työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, lkm",49301
8,1990,18-24,"Muuhun kuin työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, %",17
9,1990,25-34,"Aikuiskoulutukseen osallistuneet, lkm",416027


In [5]:
#Nyt huomataan, että dataframe on ns. pitkässä muodossa (long). Eli jokainen rivi on yksi havainto. Toisin
#sanoen kaikista mahdollisista muuttujayhdistelmistä on tehty oma rivi. Mittarit puolestaan ovat yhdessä sarakkeessa.

#Lyhyessä dataframessa (short) puolestaan jokainen rivi vastaa yhtä havaintoyksikköä ja eri mittarit ovat eri sarakkeissa. 
#Muunnetaan dataframe lyhyeen muotoon, eli pivotoidaan se. 

#Muistisääntö: Pitkä data on helppo analysoida, lyhyt data on helppo lukea. 

df = unstack(df, [:Vuosi, :Ikä], :Tiedot, :Arvo)




Row,Vuosi,Ikä,"Aikuiskoulutukseen osallistuneet, lkm","Aikuiskoulutukseen osallistuneet, %","Työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, lkm","Työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, %","Työnantajan tukemaan koulutukseen (henkilöstökoulutukseen) osallistuneet, lkm","Työnantajan tukemaan koulutukseen (henkilöstökoulutukseen) osallistuneet, %","Muuhun kuin työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, lkm","Muuhun kuin työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, %"
,String,String,Any,Any,Any,Any,Any,Any,Any,Any
1,1990,18-24,204552,43,97831,36,82445,34,49301,17
2,1990,25-34,416027,55,329104,50,285334,50,127314,18
3,1990,35-44,481846,57,373050,48,338924,52,172371,21
4,1990,45-54,291068,48,234914,44,209508,48,111111,18
5,1990,55-64,132963,25,57283,27,52524,37,85202,16
6,1995,18-24,183534,42,50555,27,37360,33,36170,18
7,1995,25-34,381866,53,260675,44,204640,50,126507,19
8,1995,35-44,425144,54,325990,45,264282,53,147087,19
9,1995,45-54,404908,54,317436,47,261089,57,131040,17


In [6]:
#Muunnetaan takaisin pitkään muotoon. 

df = stack(df, Not([:Vuosi, :Ikä]), variable_name=:Tiedot, value_name=:Arvo)





Row,Vuosi,Ikä,Tiedot,Arvo
,String,String,String,Any
1,1990,18-24,"Aikuiskoulutukseen osallistuneet, lkm",204552
2,1990,25-34,"Aikuiskoulutukseen osallistuneet, lkm",416027
3,1990,35-44,"Aikuiskoulutukseen osallistuneet, lkm",481846
4,1990,45-54,"Aikuiskoulutukseen osallistuneet, lkm",291068
5,1990,55-64,"Aikuiskoulutukseen osallistuneet, lkm",132963
6,1995,18-24,"Aikuiskoulutukseen osallistuneet, lkm",183534
7,1995,25-34,"Aikuiskoulutukseen osallistuneet, lkm",381866
8,1995,35-44,"Aikuiskoulutukseen osallistuneet, lkm",425144
9,1995,45-54,"Aikuiskoulutukseen osallistuneet, lkm",404908


In [7]:
#Analysoidaan dataa seuraavaksi tekemällä Pivot-taulukko, jossa osa datasta on suodatettu pois. 

using DataFrames

# Valitaan vuosi ja kolme muuttujaa. 
vuosi = "2022"
muuttujat = [
    "Aikuiskoulutukseen osallistuneet, %",
    "Työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, %",
    "Aikuiskoulutukseen osallistuneet, lkm"
]

#Suodatetaan vuoden ja valitun kolmen muuttujan mukaan
df_suod = filter(row -> row.Vuosi == vuosi && row.Tiedot in muuttujat, df)

#Pivotoitodaan suodatettu dataframe ja esitetään se lyhyessä muodossa: rivit = Ikä, sarakkeet = Tiedot, solut = Arvo
pivot  = unstack(df_suod, :Ikä, :Tiedot, :Arvo)   

pivot

#Tehtävä: Rivisummat ovat reilusti yli 100 %. Mieti, mistä tämä johtuu. 

Row,Ikä,"Aikuiskoulutukseen osallistuneet, lkm","Aikuiskoulutukseen osallistuneet, %","Työhön tai ammattiin liittyvään aikuiskoulutukseen osallistuneet, %"
,String,Any,Any,Any
1,18-24,178091,44,30
2,25-34,291667,48,48
3,35-44,362093,53,50
4,45-54,341665,54,52
5,55-64,279325,38,38


In [8]:
#Nyt kaikkien sarakkeiden tietotyyppi on Any. Muutetaan yksi niistä intiksi.

df_suod[!, "Aikuiskoulutukseen osallistuneet, lkm"] = Int64.(df[!, "Aikuiskoulutukseen osallistuneet, lkm"])




LoadError: ArgumentError: column name :Aikuiskoulutukseen osallistuneet, lkm not found in the data frame

In [9]:
#Saadaan virheilmoitus. Tutkitaan minkä nimisiä sarakkeita DataFramessa on. 
names(df_suod)


4-element Vector{String}:
 "Vuosi"
 "Ikä"
 "Tiedot"
 "Arvo"

In [10]:
#Aikaisemmin tekemämme muutokset ovat vaikuttaneet siis ainoastaan esittämistapaan. Dataframen sarakkeiden nimet 
#Eivät ole muuttuneet. Tehdään uusi dataframe, johon suodatetaan pelkästään Aikuiskoulutuksen osallistuneiden lukumäärän
#sisältävät arvot. 

df_aikkoul = subset(df_suod, :Tiedot => ByRow(==("Aikuiskoulutukseen osallistuneet, lkm")))




Row,Vuosi,Ikä,Tiedot,Arvo
,String,String,String,Any
1,2022,18-24,"Aikuiskoulutukseen osallistuneet, lkm",178091
2,2022,25-34,"Aikuiskoulutukseen osallistuneet, lkm",291667
3,2022,35-44,"Aikuiskoulutukseen osallistuneet, lkm",362093
4,2022,45-54,"Aikuiskoulutukseen osallistuneet, lkm",341665
5,2022,55-64,"Aikuiskoulutukseen osallistuneet, lkm",279325


In [11]:
#Muutetaan nyt sarakkeen Arvo tyypiksi int. Otetaan huomioon mahdollisuus, että sarakkeessa olisi puuttuvia
#arvoja (missing).

df_aikkoul[!, "Arvo"] = convert(Vector{Union{Missing, Int64}},
    [x === missing ? missing : Int64(x) for x in df_aikkoul[!, "Arvo"]]
)

df_aikkoul




Row,Vuosi,Ikä,Tiedot,Arvo
,String,String,String,Int64?
1,2022,18-24,"Aikuiskoulutukseen osallistuneet, lkm",178091
2,2022,25-34,"Aikuiskoulutukseen osallistuneet, lkm",291667
3,2022,35-44,"Aikuiskoulutukseen osallistuneet, lkm",362093
4,2022,45-54,"Aikuiskoulutukseen osallistuneet, lkm",341665
5,2022,55-64,"Aikuiskoulutukseen osallistuneet, lkm",279325


In [12]:
#Lasketaan lopuksi montako henkilöä osallistui aikuiskoulutukseen vuonna 2022. Laskeminen olisi onnistunut myös
#ilman tyyppimuunnosta, koska kaikki sarakkeen arvot ovat kokonaislukuja. Jos näin ei olisi, niin laskeminen antaisi
#virheilmoituksen. Myös desimaaliluvut kelpaisiva

sum(df_aikkoul.Arvo)


1452841